In [13]:
# get all noodle headlines from their wordpress site

import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

url = "https://thelocalnoodle.com/page/"


def get_headlines(page):
    r = requests.get(page)
    return parse_headlines(r.content)

def parse_headlines(html_doc):
    soup = BeautifulSoup( html_doc , 'html.parser') 
    headlines = []
    # for each entry that I find
    for e in soup.find_all("h1", {"class": "entry-title"}):
        # get the text and put it into a list
        headlines.append(e.text)
            
    return headlines
    

    
noodle_headlines = []
for i in range(1,15):
    url_i = url + str(i) + "/"
    noodle_headlines += get_headlines(url_i) 
    


In [15]:
# save it as a dataframe
df = pd.DataFrame({'text':noodle_headlines})

# Converting all text to lowercase, fixing ampersands and getting rid
# of dashes and apostrophes as they can mess up the dictionary
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'&amp;', 'and')
df['text'] = df['text'].str.replace(r'-', ' ')
df['text'] = df['text'].str.replace(r'[^\s\w]','')
df.head()

,text
0,virtue field house track converted to runway s...
1,op ed middlebury must rescind the krakens hono...
2,students instagram story of sunset wins pulitz...
3,unable to go down ski slope feb grads go down ...
4,lizard animating trumps rotting body from with...


In [16]:
df.to_csv('noodleheadlines.csv')